# Working with tables

In this notebook, we will see how to upsert a Hive table with new increment of data, we will try to make the operation more atomic and provide time-travel (roll-back) functionality

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window
import os

In [ ]:
spark = (
    SparkSession.builder.appName('tables management')
    .enableHiveSupport()
).getOrCreate()

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

users_base_path = os.path.join(project_path, 'data/users_base')
users_increment_path = os.path.join(project_path, 'data/users_increment')
accounts_output_path = os.path.join(project_path, 'output/hive/accounts')

accounts_output_path_v1 = os.path.join(project_path, 'output/tables/accounts/1')
accounts_output_path_v2 = os.path.join(project_path, 'output/tables/accounts/2')

checkpoint_dir = os.path.join(project_path, 'output/checkpoints')
tmp_location = os.path.join(project_path, 'output/tmp')

In [ ]:
spark.sql('drop table if exists accounts')

### Create a new table

* Take the data from the `users_base_path` and save it as a new table with the name `accounts`
* Use [saveAsTable](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.saveAsTable.html#pyspark.sql.DataFrameWriter.saveAsTable)
* as the location for the table use `accounts_output_path`

In [ ]:
# your code here:



### Verify that the table is created

you can use the following SQL commands:
* show tables
* describe table_name
* describe formatted table_name
* describe extended table_name

In [ ]:
# your code here:



### Upsert

* load the increment in to a Spark DataFrame 
 * use the path `users_increment_path`
* upsert the increment on the accounts table
 * use the approach with Union + row_number:
   * add a new column `version` to both dataframes, use value 1 for the table and value 2 for the increment
   * union both DataFrames using [unionByName](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.unionByName.html#pyspark.sql.DataFrame.unionByName)
   * create a [window](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.WindowSpec.partitionBy.html#pyspark.sql.WindowSpec.partitionBy) partitioned by user_id and sorted by the new `version` column
   * call [row_number](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.row_number.html#pyspark.sql.functions.row_number) over this window
   * this will allow you to use a filter to keep for each `user_id` only records with newer `version`


In [ ]:
# read the increment:



In [ ]:
# define the window:



In [ ]:
# Write the query for the upsert - create a new dataframe called `result`:



#### Save the result

Try to run the overwrite of the `accounts` table by this `result` DataFrame. 

Notice that running the overwrite will lead to the following error:

`AnalysisException: Cannot overwrite table default.accounts that is also being read from`

This is because we cannot write to the same location from which we also read

In [ ]:
# run the overwrite to see the error:



### Checkpointing

This can be solved using checkpointing

* Checkpoint the result DataFrame using [checkpoint](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.checkpoint.html#pyspark.sql.DataFrame.checkpoint)
* assign it to a new DataFrame
* run the overwrite with this new checkpointed DataFrame

Note:
* the checkpoint will persist the data at a location specified using `setCheckpointDir`

In [ ]:
spark.sparkContext.setCheckpointDir(checkpoint_dir)

In [ ]:
# do the checkpoint:



In [ ]:
# save the checkpointed result - the error should no longer be present:



### Atomicity

If the saving process fails from some reason you may end up with a corrupted table. To avoid that, try to make the process more atomic. Do the saving again as follows:

1. Crecreate the original table `accounts` at a new location (use `accounts_output_path_v1`)
2. Do the upsert and save it at a different location, namaly accounts_output_path_v2, use a different name for the final table, namely `accounts_v2`
3. Use SQL command `ALTER TABLE` to rename the `accounts` table to `accounts_delete`
4. Use `ALTER TABLE` again to rename the `account_v2` to `accounts`
5. Use SQL command `DROP TABLE` to delete `accounts_delete`

Basicly, you will first write the result and after it is successfully written, you will switch the table names to make sure that your production table is still in a consistent state. On the other hand, if your write would fail from some reason, you woudn't make the switch to keep the original table in consistent state.

In [ ]:
# Resave the original table at the location accounts_output_path_v1:


In [ ]:
# Do the upsert - save the result at the location accounts_output_path_v2, use a new table_name (accounts_v2):



In [ ]:
# Run the SQL commands to switch the names:



In [ ]:
# drop the original table (the one that was renamed to accounts_delete):



### Time Travel

Now imagine, that you have made a mistake and you actually don't want to do the upsert. We want to roll-back the operation. We can do it because the `DROP` command didn't delete the actual data, but only removed the information from the metastore. We can reconstruct the original data back so long we have the data and now the schema.

1. Create an empty DataFrame with the schema of the accounts table (use the schema of the new table, because we didn't change it). To create an empty DataFrame use [createDataFrame](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.createDataFrame.html#pyspark.sql.SparkSession.createDataFrame)
2. Save the empty DataFrame at temporal location - use `tmp_location`
3. Use ALTER TABLE command to change the location so the table points to the data before the upsert - `accounts_output_path_v1`
4. Now the table is no longer empty so you can switch the names using ALTER TABLE to give it the proper name


In [ ]:
# create empty DataFrame:



In [ ]:
# save it as an empty table at temporal location:



In [ ]:
# Change the location of the empty table and switch the names
# Drop the table with the wrong upsert



In [ ]:
spark.stop()